In [1]:
import sys
import os
import random
import torch
import numpy as np
import itertools
import pandas as pd
import csv
from tqdm import tqdm
from box import Box

# Ajustar path al proyecto
project_root = "/home/javitrucas/TFG"
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.dataset_loader import load_dataset
from scripts.medical_scripts.medical_training import Training
from scripts.medical_scripts.medical_evaluation import ModelEvaluator
from scripts.MIL_utils import MIL_collate_fn

def run_experiment(config: Box):
    # Fijar semillas
    seed = config.seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # Crear directorio de salida
    out_dir = f"./models/{config.dataset_name.split('-')[0]}/{config.pooling_type}/seed{seed}"
    os.makedirs(out_dir, exist_ok=True)

    # Cargar datos
    train_ds, val_ds = load_dataset(config=config, mode="train_val")
    test_ds          = load_dataset(config=config, mode="test")
    train_loader = torch.utils.data.DataLoader(
        train_ds, batch_size=config.batch_size, shuffle=True, collate_fn=MIL_collate_fn
    )
    val_loader = torch.utils.data.DataLoader(
        val_ds, batch_size=config.batch_size, shuffle=False, collate_fn=MIL_collate_fn
    )
    test_loader = torch.utils.data.DataLoader(
        test_ds, batch_size=config.batch_size, shuffle=False, collate_fn=MIL_collate_fn
    )

    # Entrenamiento
    trainer = Training(
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=config.num_epochs,
        learning_rate=config.learning_rate,
        output_model_dir=out_dir,
        pooling_type=config.pooling_type,
        input_feature_dim=config.input_feature_dim,
        feature_dim=config.feature_dim,
        wandb=None
    )
    trainer.train()

    # Épocas entrenadas (ajusta si tu Training usa otro atributo)
    trained_epochs = getattr(trainer, "current_epoch", config.num_epochs)

    # Guardar modelo
    model_path = os.path.join(out_dir, "model.pth")
    torch.save(trainer.model.state_dict(), model_path)

    # Evaluación en train
    evaluator_train = ModelEvaluator(
        model_path=model_path,
        test_loader=train_loader,
        batch_size=config.batch_size,
        input_feature_dim=config.input_feature_dim,
        feature_dim=config.feature_dim,
        pooling_type=config.pooling_type,
        wandb=None
    )
    train_metrics, _ = evaluator_train.evaluate()
    print(f"Train AUC: {train_metrics.get('auc')}")

    # Evaluación en test
    evaluator_test = ModelEvaluator(
        model_path=model_path,
        test_loader=test_loader,
        batch_size=config.batch_size,
        input_feature_dim=config.input_feature_dim,
        feature_dim=config.feature_dim,
        pooling_type=config.pooling_type,
        wandb=None
    )
    test_metrics, _ = evaluator_test.evaluate()
    print(f"Test AUC: {test_metrics.get('auc')}")

    # Devolver métricas + épocas
    return {
        "trained_epochs": trained_epochs,
        **{f"train_{k}": v for k, v in train_metrics.items()},
        **{f"test_{k}":  v for k, v in test_metrics.items()}
    }


def run_all_experiments():
    csv_file = "PANDA_experiment_runs.csv"
    header = [
        "dataset", "pooling", "learning_rate", "feature_dim", "seed",
        "trained_epochs",
        # métricas de train
        "train_loss", "train_accuracy", "train_f1_score", "train_auc", "train_precision", "train_recall",
        # métricas de test
        "test_loss",  "test_accuracy",  "test_f1_score",  "test_auc",  "test_precision",  "test_recall"
    ]

    # Si existe un CSV previo, lo cargamos y construimos el set de combinaciones ya hechas
    if os.path.exists(csv_file):
        df_done = pd.read_csv(csv_file)
        print("Últimos experimentos ejecutados:")
        display(df_done.tail())
        done_set = set(zip(
            df_done.dataset, df_done.pooling,
            df_done.learning_rate, df_done.feature_dim, df_done.seed
        ))
    else:
        # Creamos CSV con cabecera
        with open(csv_file, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(header)
        done_set = set()
        print("No se encontró CSV previo. Empezando desde cero.")

    # Rejilla de hiperparámetros
    datasets       = ["panda-patches_512_preset-features_UNI"]
    pooling_types  = ["attention", "mean", "max"]
    learning_rates = [1e-3, 1e-4]
    feature_dims   = [64, 128]
    seeds          = list(range(5))

    for dataset, pooling, lr, feat_dim in tqdm(
        itertools.product(datasets, pooling_types, learning_rates, feature_dims),
        desc="Grid search PANDA"
    ):
        for seed in seeds:
            key = (dataset, pooling, lr, feat_dim, seed)
            if key in done_set:
                continue  # ya ejecutado

            # Parámetros dinámicos: para PANDA usamos 1024, para otros 512
            input_feat_dim = 1024 if "panda" in dataset else 512
            num_epochs     = 50   if "panda" in dataset else 15

            config = Box({
                "dataset_name":      dataset,
                "input_feature_dim": input_feat_dim,
                "feature_dim":       feat_dim,
                "pooling_type":      pooling,
                "num_epochs":        num_epochs,
                "learning_rate":     lr,
                "batch_size":        1,
                "val_prop":          0.2,
                "seed":              seed,
                "use_inst_distances": False,
                "adj_mat_mode":      "relative"
            })

            print(f"\n=== Ejecutando: {dataset}, {pooling}, lr={lr}, feat_dim={feat_dim}, seed={seed} ===")
            metrics = run_experiment(config)

            # Preparar la fila para CSV
            row = [
                dataset, pooling, lr, feat_dim, seed,
                metrics.get("trained_epochs", ""),
                # train
                metrics.get("train_loss", ""),
                metrics.get("train_accuracy", ""),
                metrics.get("train_f1_score", ""),
                metrics.get("train_auc", ""),
                metrics.get("train_precision", ""),
                metrics.get("train_recall", ""),
                # test
                metrics.get("test_loss", ""),
                metrics.get("test_accuracy", ""),
                metrics.get("test_f1_score", ""),
                metrics.get("test_auc", ""),
                metrics.get("test_precision", ""),
                metrics.get("test_recall", ""),
            ]

            # Guardado inmediato al CSV
            with open(csv_file, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(row)

            done_set.add(key)

    print("\nTodos los experimentos quedan registrados en", csv_file)


if __name__ == "__main__":
    run_all_experiments()


/home/javitrucas/miniconda3/envs/tfg/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/javitrucas/miniconda3/envs/tfg/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Últimos experimentos ejecutados:


,dataset,pooling,learning_rate,feature_dim,seed,trained_epochs,train_loss,train_accuracy,train_f1_score,train_auc,train_precision,train_recall,test_loss,test_accuracy,test_f1_score,test_auc,test_precision,test_recall
49,panda-patches_512_preset-features_UNI,max,0.0010,128,4,50,NaN,0.952077,0.966234,0.989956,0.992815,0.941039,NaN,0.924192,0.945294,0.976172,0.988225,0.905937
50,panda-patches_512_preset-features_UNI,max,0.0001,64,0,50,NaN,0.967957,0.977606,0.993152,0.995962,0.959914,NaN,0.949833,0.964455,0.979054,0.988664,0.941403
51,panda-patches_512_preset-features_UNI,max,0.0001,64,1,50,NaN,0.986531,0.990698,0.997886,0.997044,0.984433,NaN,0.950390,0.964808,0.983516,0.990260,0.940632
52,panda-patches_512_preset-features_UNI,max,0.0001,64,2,50,NaN,0.982844,0.988103,0.996060,0.998609,0.977817,NaN,0.952620,0.966575,0.982690,0.986356,0.947571
53,panda-patches_512_preset-features_UNI,max,0.0001,64,3,50,NaN,0.988374,0.991975,0.999192,0.997834,0.986184,NaN,0.950948,0.965327,0.983177,0.987107,0.944487


Grid search PANDA: 0it [00:00, ?it/s]


=== Ejecutando: panda-patches_512_preset-features_UNI, max, lr=0.0001, feat_dim=64, seed=4 ===
panda
[WSIDataset] Scanning files...


...

[WSIDataset] Skipped 0 bags
[WSIDataset] Found 1794 already processed bags
Epoch 1/50
Train - Loss: 0.1884, Acc: 0.9207, AUC: 0.9723, F1: 0.9453
Val   - Loss: 0.1675, Acc: 0.9450, AUC: 0.9815, F1: 0.9612
Final model saved to ./models/panda/max/seed4/model_max.pth
Epoch 2/50
Train - Loss: 0.1218, Acc: 0.9517, AUC: 0.9880, F1: 0.9665
Val   - Loss: 0.1511, Acc: 0.9484, AUC: 0.9811, F1: 0.9641
Epoch 3/50
Train - Loss: 0.1074, Acc: 0.9606, AUC: 0.9906, F1: 0.9728
Val   - Loss: 0.1471, Acc: 0.9490, AUC: 0.9821, F1: 0.9644
Final model saved to ./models/panda/max/seed4/model_max.pth
Epoch 4/50
Train - Loss: 0.0926, Acc: 0.9658, AUC: 0.9927, F1: 0.9764
Val   - Loss: 0.1513, Acc: 0.9478, AUC: 0.9805, F1: 0.9638
Epoch 5/50
Train - Loss: 0.0752, Acc: 0.9712, AUC: 0.9954, F1: 0.9802
Val   - Loss: 0.1563, Acc: 0.9461, AUC: 0.9834, F1: 0.9622
Final model saved to ./models/panda/max/seed4/model_max.pth
Epoch 6/50
Train - Loss: 0.0630, Acc: 0.9792, AUC: 0.9965, F1: 0.9856
Val   - Loss: 0.2103, Acc: 0.9

/home/javitrucas/TFG/scripts/medical_scripts/medical_evaluation.py:64: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + ["Negative", "Positive"])
/home/javitrucas/TFG/scripts/medical_scripts/medical_evaluation.py:65: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + ["Negative", "Positive"])


Confusion matrix saved at output/max/confusion_matrix.png

--- Evaluation Results ---
Optimal Threshold: 0.4866
Accuracy: 0.9823, AUC: 0.9957
Precision: 0.9974, Recall: 0.9782, F1-Score: 0.9877
Confusion Matrix:
[[1901   13]
 [ 112 5027]]
Train AUC: 0.9956520130141724
Model loaded successfully from ./models/panda/max/seed4/model.pth


/home/javitrucas/TFG/scripts/medical_scripts/medical_evaluation.py:64: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + ["Negative", "Positive"])
/home/javitrucas/TFG/scripts/medical_scripts/medical_evaluation.py:65: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + ["Negative", "Positive"])
Grid search PANDA: 12it [1:29:39, 448.29s/it]

Confusion matrix saved at output/max/confusion_matrix.png

--- Evaluation Results ---
Optimal Threshold: 0.6844
Accuracy: 0.9476, AUC: 0.9750
Precision: 0.9831, Recall: 0.9437, F1-Score: 0.9630
Confusion Matrix:
[[ 476   21]
 [  73 1224]]
Test AUC: 0.9749600145204304

Todos los experimentos quedan registrados en PANDA_experiment_runs.csv


In [3]:
import pandas as pd
import numpy as np

# Cargar el CSV
df = pd.read_csv('PANDA_experiment_runs.csv')

# Definir las métricas para calcular estadísticas
metrics_to_analyze = [
    'train_accuracy', 'train_f1_score', 'train_auc', 'train_precision', 'train_recall',
    'test_accuracy', 'test_f1_score', 'test_auc', 'test_precision', 'test_recall'
]

# Agrupar por hiperparámetros (excluyendo seed)
groupby_cols = ['dataset', 'pooling', 'learning_rate', 'feature_dim']

# Calcular estadísticas agrupadas
results_summary = []

for group_name, group_data in df.groupby(groupby_cols):
    dataset, pooling, lr, feature_dim = group_name
    
    # Crear diccionario base para este grupo
    row = {
        'dataset': dataset,
        'pooling': pooling,
        'learning_rate': lr,
        'feature_dim': feature_dim,
        'n_seeds': len(group_data)
    }
    
    # Calcular medias y desviaciones para cada métrica
    for metric in metrics_to_analyze:
        if metric in group_data.columns:
            values = group_data[metric].dropna()
            if len(values) > 0:
                row[f'{metric}_mean'] = values.mean()
                row[f'{metric}_std'] = values.std() if len(values) > 1 else 0.0
    
    results_summary.append(row)

# Convertir a DataFrame
df_clean = pd.DataFrame(results_summary)

# Guardar CSV limpio
df_clean.to_csv('PANDA_clean.csv', index=False, float_format='%.6f')

print(f"CSV limpio guardado en 'PANDA_clean.csv' con {len(df_clean)} configuraciones")

CSV limpio guardado en 'PANDA_clean.csv' con 12 configuraciones
